In [1679]:
import numpy as np
import pandas as pd
import random as rd
from collections import defaultdict

Structure of overall applications.

Get data ready and split into two groups. Group 1 is prop on the impromptu motion on Day 1 and Group 2 is opp on the prepped motion of Day 1. On Day 2, Group 1 is opp on the impromptu and Group 2 is Prop on the impromptu. The alternation continues on Day 3 and Day 4.

Then the same process is applied for each day. First, pairs are formed between countries within groups. The pairs from both groups get paired into 2x2 blocks. 

Day 1 to 4, with the sides that groups take alternating
Pairs (for both groups)
Blocks (update penalty scores)


In [1680]:
rankings = pd.read_csv('WSDCrankings.csv', sep=';')
rankings['penaltyStrength'] = pd.Series(np.repeat(0,62), index=rankings.index) # Create penalty columns
rankings['penaltyPowerdif'] = pd.Series(np.repeat(0,62), index=rankings.index)
rankings = rankings[:60]
rankings

,nation,strength,penaltyStrength,penaltyPowerdif
0,Australia,0.0689,0,0
1,South Africa,0.0859,0,0
2,Canada,0.0955,0,0
3,England,0.1166,0,0
4,Singapore,0.1300,0,0
5,Ireland,0.1369,0,0
6,Hong Kong,0.2049,0,0
7,Greece,0.2196,0,0
8,Pakistan,0.2285,0,0
9,Malaysia,0.2307,0,0


In [1681]:
rankings.count()

nation             60
strength           60
penaltyStrength    60
penaltyPowerdif    60
dtype: int64

In [1682]:
group1 = rankings['nation'][range(0,59,2)].as_matrix()
group2 = rankings['nation'][range(1,60,2)].as_matrix()
group1

array(['Australia', 'Canada', 'Singapore', 'Hong Kong', 'Pakistan',
       'Swaziland', 'New Zealand', 'India', 'Slovenia', 'Mexico',
       'Germany', 'Wales', 'Indonesia', 'Sri Lanka', 'Thailand',
       'South Korea', 'Barbados', 'Qatar', 'Bermuda', 'Romania',
       'Bangaldesh', 'Nepal', 'Palestine', 'Montenegro', 'Mongolia',
       'Hungary', 'Bosnia & Herzegovina', 'Zimbabwe', 'Morocco', 'Uganda'], dtype=object)

In [1683]:
# Set up dict to remember what nations have faced on another.
nations = rankings['nation'].as_matrix()
matchUps = defaultdict(list)
for nation in nations:
    matchUps[nation].append('land')

In [1684]:
averageStrength = rankings['strength'].mean()
averageStrength

0.5538633333333333

In [1685]:
def generatePairs(group):

    # Pair countries based on two criteria: strength and penalty. 
    # Iterate through countries in the group.
    # Take the sum of the difference between the strength and penalty for each combination.
    # Choose partner with lowest sum.
    # Put paired countries on list that are no longer available for partnering.
    # Repeat.
    
    # Create a table to store all the matches in.
    pairTable = pd.DataFrame(columns=('nation', 'pairno','groupno'))
    pairCount = 0
    
    # If the group I am matching is group 2, I remove group 1 countries.
    if group is group2: 
        pairCount += 100 # We should be able to identify pairs from group 2. They are pairs with an id > 100.
        groupno = 2
        # Below does the same as a for loop but is more efficient.
        countrieslist = [x for x in rankings['nation'].as_matrix().tolist() if x not in group1]

    else:
        groupno = 1
        # Same as above
        countrieslist = [x for x in countries.tolist() if x not in group2]


    for countryOne in group:

        if countryOne in countrieslist: # If country was already matched, can't match again.
            # Remove now, so it is not matched with self and is not matched again.
            countrieslist.remove(countryOne) 
            pairCount += 1 # Get ready to match a pair with this number
            
            
            
            # Filter the table for countries still requiring a matching
            matchableCountries = rankings.loc[rankings['nation'].isin(countrieslist)]

            # Compile a score of strength match, strength penalty and powerdif penalty.
            
            # Following the logic that the Average strength of a pair is the pair's scores divided by two we can find the 
            # country that brings the pair closest to the average.
            bestMatchStrength = 2 * averageStrength - rankings['strength'].loc[rankings['nation'] == countryOne].values[0]
            strengthError = np.abs(bestMatchStrength - matchableCountries['strength'].values)
            
            bestPenaltyPowerdif = rankings['penaltyPowerdif'].loc[rankings['nation'] == countryOne].values[0]
            penaltyPowerdifError = np.abs(bestPenaltyPowerdif - matchableCountries['penaltyPowerdif'].values)
            
            bestPenaltyStrength = rankings['penaltyStrength'].loc[rankings['nation'] == countryOne].values[0]
            penaltyStrengthError = np.abs(bestPenaltyStrength - matchableCountries['penaltyStrength'].values)
            
            penaltyTotal = strengthError + penaltyPowerdifError + penaltyStrengthError
            
            indexOfBestMatch = np.argmin(penaltyTotal) # Get index of lowest penalty score.
            
            bestMatchedCountries = [matchableCountries['nation'].iloc[indexOfBestMatch]]
            bestMatchedCountry = bestMatchedCountries[rd.randint(0,len(bestMatchedCountries) - 1)]
            countrieslist.remove(bestMatchedCountry)
            
            # Power differential is the difference between the strenghts of the paired nations.
            bestMatchedCountryStrength = rankings['strength'].loc[rankings['nation'] == bestMatchedCountry].values
            countryOneStrenght = rankings['strength'].loc[rankings['nation'] == countryOne].values[0]
            powerdif = np.abs(bestMatchedCountryStrength - countryOneStrenght)
            
            newPairing = pd.DataFrame({'nation': [countryOne, bestMatchedCountry],
                                       'pairno': [pairCount, pairCount],
                                       'groupno': [groupno, groupno],
                                         'powerdif': [powerdif, powerdif]})
            pairTable = pd.concat([pairTable, newPairing])
            
    return(pairTable)

In [1686]:
def generateBlocks(pairTable):
    attributesTable = pd.DataFrame(columns=('pairno', 'strength','penalty', 'groupno')) # Table to collect the attributes data.
    pairNumbers = np.unique(pairTable['pairno'].values) # Get all unique pairs.
    averagePowerdif = pairTable['powerdif'].mean()
    
    nedPair = pairTable['pairno'].loc[pairTable['nation'] == 'Bangaldesh'].values
    
    for pairNumber in pairNumbers:
        pairCountries = pairTable['nation'].loc[pairTable['pairno'] == pairNumber].values # Get the two countries in pair.
        meanStrength = np.mean(rankings['strength'].loc[rankings['nation'].isin(pairCountries)])
        meanPenaltyStrength = np.mean(rankings['penaltyStrength'].loc[rankings['nation'].isin(pairCountries)])
        meanPenaltyPowerdif = np.mean(rankings['penaltyPowerdif'].loc[rankings['nation'].isin(pairCountries)])
        powerdif = np.mean(pairTable['powerdif'].loc[pairTable['pairno'] == pairNumber])
        groupNo = pairTable['groupno'].loc[pairTable['pairno'] == pairNumber].values[0]
        
        pairAttributes = pd.DataFrame({'pairno': [pairNumber],
                                       'strength': [meanStrength],
                                       'powerdif': [powerdif],
                                       'groupno': [groupNo],
                                       'penaltyStrength': [meanPenaltyStrength],
                                       'penaltyPowerdif': meanPenaltyPowerdif,
                                      })
        attributesTable = pd.concat([attributesTable, pairAttributes]) # Add results to attributes tables.
    
    # End of for loop.
    # Let's make some blocks!
    unmatchedPairs = pairNumbers.tolist() # List of all pair numbers, because all remain to be matched.
    blockTable = pd.DataFrame(columns=('pairno', 'blockno','penaltyStrength', 'penaltyPowerdif')) # Table with all the blocks.
    blockCount = 0
    attributesTable = attributesTable.sort_values(by = "penaltyPowerdif")
    pairNumbers = attributesTable['pairno'].values
    
    for pairNumber in pairNumbers:
        
        if pairNumber in unmatchedPairs:

            # Get data necessary to sort legal matches.
            groupNo = attributesTable['groupno'].loc[attributesTable['pairno'] == pairNumber].values[0]
            unmatchedPairs.remove(pairNumber) # Remove so pair doesn't match itself.
            blockCount += 1 # Set id for new block.
            
            # Statement checks if the group is from the other group and filters out already matched pairs.
            matchablePairs = attributesTable.loc[(attributesTable['groupno'] != groupNo) & 
                                                 (attributesTable['pairno'].isin(unmatchedPairs))]
            
            # Also filter out pairs with countries already have been seen.
            pairCountries = pairTable['nation'].loc[pairTable['pairno'] == pairNumber].values
            countriesAlreadyMet = [matchUps[pairCountries[0]], matchUps[pairCountries[1]]]
            countriesAlreadyMet = [item for sublist in countriesAlreadyMet for item in sublist]
            countriesAlreadyMet = [item for sublist in countriesAlreadyMet for item in sublist]
            clashPairs = pairTable['pairno'].loc[pairTable['nation'].isin(list(countriesAlreadyMet))].values
            
            matchablePairs = matchablePairs.loc[~matchablePairs['pairno'].isin(clashPairs)]
        
            # Vector with absolute error margin strength
            pairStrength = attributesTable['strength'].loc[attributesTable['pairno'] == pairNumber].values[0]
            bestMatchStrength = (2 * averageStrength - pairStrength) + attributesTable['penaltyStrength'].loc[attributesTable['pairno'] == pairNumber].values[0]
            strengthError = bestMatchStrength - matchablePairs['strength'].values
            
            # Vector with absolute error margin powerdif
            pairPowerdif = attributesTable['powerdif'].loc[attributesTable['pairno'] == pairNumber].values[0]
            bestMatchPowerdif = averagePowerdif + attributesTable['penaltyPowerdif'].loc[attributesTable['pairno'] == pairNumber].values
            powerdifError = bestMatchPowerdif - matchablePairs['powerdif'].values
            
            # Also determine quality of match 'the other way around'
            for otherPair in matchablePairs['pairno'].values
            
                otherPairPowerdif = attributesTable['powerdif'].loc[attributesTable['pairno'] == otherPair].values[0]
                otherBestMatchPowerdif = averagePowerdif + attributesTable['penaltyPowerdif'].loc[attributesTable['pairno'] == otherPair].values
                otherPowerdifError = otherBestMatchPowerdif - pairPowerdif
                
                otherPairStrength = attributesTable['strength'].loc[attributesTable['pairno'] == otherPair].values[0]
                otherBestMatchStrength = (2 * averageStrength - otherPairStrength) + attributesTable['penaltyStrength'].loc[attributesTable['pairno'] == otherPair].values[0]
                otherStrengthError = otherBestMatchStrength - pairStrength
                
            # Vector with overall penalty score
            penaltyScore = np.abs(strengthError) + np.abs(powerdifError) + np.abs(otherPowerdifError) + np.abs(otherStrengthError)
            
            try:
                indexOfBestMatchInMatchablePairs = np.argmin(penaltyScore) # Get index of lowest penalty score.
            except(ValueError):
                print(pairNumbers)
            
            try:
                bestMatchedPair = matchablePairs['pairno'].iloc[indexOfBestMatchInMatchablePairs]
            except(IndexError):
                print(matchablePairs)
            
            bestMatchedStrength = matchablePairs['strength'].loc[matchablePairs['pairno'] == bestMatchedPair].values
            bestMatchedPowerdif = matchablePairs['powerdif'].loc[matchablePairs['pairno'] == bestMatchedPair].values
            unmatchedPairs.remove(bestMatchedPair)
            
            if pairNumber == nedPair:
                print(bestMatchedPowerdif.tolist() - bestMatchPowerdif)
            elif bestMatchedPair == nedPair:
                print(pairPowerdif.tolist() - bestMatchPowerdif)
            
            # Calculate the penalty scores for strength and powerdif.
            penaltyPairStrength = bestMatchStrength - bestMatchedStrength # Strength
            penaltyMatchedPairStrength = averageStrength - pairStrength
            
            penaltyPairPowerdif = bestMatchPowerdif - bestMatchedPowerdif  # Powerdif
            penaltyMatchedPairPowerdif = averagePowerdif - attributesTable['powerdif'].loc[attributesTable['pairno'] == pairNumber].values
            
            block1 = pd.DataFrame({'pairno': [pairNumber],
                                                  'blockno': blockCount,
                                                  'penaltyStrength': penaltyPairStrength.item(0),
                                                   'penaltyPowerdif':penaltyPairPowerdif.item(0)
                                  })
            
            block2 = pd.DataFrame({'pairno': [bestMatchedPair],
                                                  'blockno': blockCount,
                                                  'penaltyStrength': penaltyMatchedPairStrength,
                                                   'penaltyPowerdif': penaltyMatchedPairPowerdif.item(0)
                                  })
            
            blockTable = pd.concat([blockTable, block1, block2])
    
    return blockTable

In [1687]:
def updatePenalties(blockTable, pairTable):
    
    # Fetch PairNumber
    for pairNumber in pairTable['pairno'].unique():
        
        # Get countries in pair
        countryOne = pairTable['nation'].loc[pairTable['pairno'] == pairNumber].values[0]
        countryTwo = pairTable['nation'].loc[pairTable['pairno'] == pairNumber].values[1]
        
        # Get pair penalty updates
        strengthUpdate = blockTable['penaltyStrength'].loc[blockTable['pairno'] == pairNumber].values[0]
        powerdifUpdate = blockTable['penaltyPowerdif'].loc[blockTable['pairno'] == pairNumber].values[0]
        
        for country in [countryOne, countryTwo]:
            
            countryIndex = rankings.loc[rankings['nation'] == country].index
            
            strengthUpdated = rankings['penaltyStrength'].loc[rankings['nation'] == country] + strengthUpdate
            rankings.set_value(countryIndex, 'penaltyStrength', strengthUpdated)
            
            powerdifUpdated = rankings['penaltyPowerdif'].loc[rankings['nation'] == country] + powerdifUpdate
            rankings.set_value(countryIndex, 'penaltyPowerdif', powerdifUpdated)
            
            if country == 'Australia':
                print()
    
    # Don't forget to sort by penalty at the end!

In [1688]:
def generateDayDraw(dayno): # This function will return a table with the matchups of the day
    
    pairs1 = generatePairs(group1) # Generate pairs within each group.
    pairs2 = generatePairs(group2)
    pairTable = pd.concat([pairs1,pairs2]) # Concat into one pairTable
    blockTable = generateBlocks(pairTable) # Create blocks.
    updatePenalties(blockTable, pairTable) # Update penalties based on blocks.
    rankings.sort_values(by = 'penaltyPowerdif')
    
    # Create matchups from blocks
    dayDraw = pd.DataFrame(columns=('round', 'prop','opp'))
    
    if dayno == 1:
        dayFirstRound = 1
        daySecondRound = 2
    elif dayno == 2:
        dayFirstRound = 3
        daySecondRound = 4
    elif dayno == 3:
        dayFirstRound = 5
        daySecondRound = 6
    elif dayno == 4:
        dayFirstRound = 7
        daySecondRound = 8
    else:
        print("No correct day specified. Please try again!")
    
    for blockNo in np.unique(blockTable['blockno'].values):
        pairs = blockTable['pairno'].loc[blockTable['blockno'] == blockNo].values
        group1Pair = pairs[pairs < 100]
        group2Pair = pairs[pairs > 100]
        
        group1CountryOne = pairTable['nation'].loc[pairTable['pairno'] == group1Pair[0]].values[0]
        group1CountryTwo = pairTable['nation'].loc[pairTable['pairno'] == group1Pair[0]].values[1]
        
        group2CountryOne = pairTable['nation'].loc[pairTable['pairno'] == group2Pair[0]].values[0]
        group2CountryTwo = pairTable['nation'].loc[pairTable['pairno'] == group2Pair[0]].values[1]
        
        blockCountries = [group1CountryOne,group1CountryTwo,group2CountryOne,group2CountryTwo]
    
        # Countries from group 1 are prop in the first round and opp in the second round on day 1 and day 3.
        if dayno == 1 or dayno == 3:
            blockDraw = pd.DataFrame({'round': [dayFirstRound, dayFirstRound, daySecondRound, daySecondRound],
                                                      'prop': [group1CountryOne, group1CountryTwo, group2CountryTwo, group2CountryOne],
                                                      'opp': [group2CountryOne, group2CountryTwo, group1CountryOne, group1CountryTwo]
                                      })
        elif dayno == 2 or dayno == 4:
            blockDraw = pd.DataFrame({'round': [dayFirstRound, dayFirstRound, daySecondRound, daySecondRound],
                                                      'prop': [group2CountryOne, group2CountryTwo, group1CountryTwo, group1CountryOne],
                                                      'opp': [group1CountryOne, group1CountryTwo, group2CountryOne, group2CountryTwo]
                                      })
    
        dayDraw = pd.concat([dayDraw, blockDraw])
        
        # Add countries to the list of 'seen' for next rounds!
        for country in [group1CountryOne,group1CountryTwo]:
            matchUps[country].append([group2CountryOne,group2CountryTwo])
        
        for country in [group2CountryOne,group2CountryTwo]:
            matchUps[country].append([group1CountryOne,group1CountryTwo])
    
    return dayDraw.sort_values(by = 'round')
    

In [1689]:
def generateDraw():
    matchedPowerdifs = []
    draw = pd.DataFrame(columns=('round', 'prop','opp'))
    
    for day in range(1,5):
        dayDraw = generateDayDraw(day)
        draw = pd.concat([draw,dayDraw])
    
    return draw

In [1690]:
draw = generateDraw()
pd.concat([draw.loc[draw['prop'] == 'Bangaldesh'], draw.loc[draw['opp'] == 'Bangaldesh']]).sort_values('round')

[ 0.16590667]
()
[ 0.74721333]
()
[[ 0.27142]]
()
[[ 0.67724]]
()


,opp,prop,round
1,Bahrain,Bangaldesh,1
3,Bangaldesh,Malaysia,2
1,Bangaldesh,Rwanda,3
2,South Africa,Bangaldesh,4
1,Israel,Bangaldesh,5
3,Bangaldesh,Philippines,6
1,Bangaldesh,Slovakia,7
2,Estonia,Bangaldesh,8


In [1691]:
matchUps['Bangaldesh']

['land',
 ['Malaysia', 'Bahrain'],
 ['South Africa', 'Rwanda'],
 ['Philippines', 'Israel'],
 ['Estonia', 'Slovakia']]

In [1692]:
rankings.sort_values(by = "penaltyStrength", ascending = False)

,nation,strength,penaltyStrength,penaltyPowerdif
59,Rwanda,1.0000,0.411550,-0.670000
1,South Africa,0.0859,0.411550,-0.670000
4,Singapore,0.1300,0.369743,-1.544600
54,Zimbabwe,0.9057,0.369743,-1.544600
45,Sweden,0.7882,0.361093,-1.335700
13,Peru,0.2935,0.361093,-1.335700
32,Barbados,0.6122,0.279143,-1.420800
22,Wales,0.4373,0.279143,-1.420800
47,Taiwan,0.8285,0.144850,-0.357900
11,Netherlands,0.2769,0.144850,-0.357900


In [1693]:
0.3528 - 0.47

-0.11719999999999997

In [1694]:
0.8704 - 0.47

0.4004

In [1695]:
0.3001 - 0.47

-0.1699

In [1696]:
-0.11719999999999997 + 0.4004 + -0.1699

0.11330000000000001